# Librerias

In [1]:
#importar las librerias necesarias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
from google.colab import files
import plotly.graph_objects as go
from datetime import datetime 
import pandas as pd
from scipy.stats import norm
from ipywidgets import interact  
from IPython.display import display
import ipywidgets as widgets

# Carga de los datos

In [2]:
# se cargan los datasts de los archivos especificados
poblaciones = pd.read_csv('/content/1.Poblaciones.csv', sep = ';', encoding='latin1')
personasSect = pd.read_csv('/content/4. Sectorización del empleo.csv', sep = ';', encoding='latin1')
dataHist = pd.read_csv('/content/1.Poblaciones.csv', sep = ';', encoding='latin1')

# Empleados y desempleados

## Preprocesamiento

In [4]:
# Se crea la estructura de un nuevo dataset para realizar las graficas 
# el numero dataset tendra las columnas Fecha, Empleado, Desempleado
""" 
Fecha   Empleado   Desempleado
dic-07  6214125    561234        --> las filas representan el numero de empleados y desempleados
                                      en una fecha en especifico.

"""

df = pd.DataFrame()
df['Fecha'] = None
df = df.assign(Empleado=None)
df = df.assign(Desempleado=None)


empleo='Empleo'
desempleo='Desempleo'

empleados=[]
desempleado=[]
fechas=[]

auxfecha='07-dic'
sumaEmpleo=0
sumaDesempleado=0

fechascomparacion=poblaciones['Unnamed: 1']
fechasLista = []

#Se liminan las fechas repetidas
for element in fechascomparacion:
    if element not in fechasLista:
        fechasLista.append(element)

del fechasLista[0]       

In [5]:
#Se elimina la primera fila ya que contiene un valor nulo
del fechasLista[0]

In [8]:
#funcion para ordenar las fechas segun el orden de los meses del año
def ordenar_fechas(fechasList):
  lista=['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
  nuevasFechas=[]
  for i in lista:
    for j in fechasList:
      if i==j[0:3]:
        nuevasFechas.append(j)

  return nuevasFechas      

fechasList = ordenar_fechas(fechasLista)


In [9]:
import re
# funcion para eliminar los puntos de un string por ejemplo 6.213.214 =6213214 
# una vez eliminados los puntos se puede convertir el string en un valor floar
# para realizar los calculos pertinentes.
def eliminarPuntos(numero):
  out = re.sub(r'[^\w\s]','',numero)
  return out

In [10]:
''' 
Se crean dos arreglos con el numero de empleados y desmpleados 
segun las fechas ordenadas

'''
for j in fechasList:
  sumaEmpleo=0
  sumaDesempleado=0
  for i in poblaciones.index:
    if str(poblaciones['Unnamed: 1'][i]) == j:

        if str(poblaciones['Unnamed: 2'][i]) == empleo:

          sumaEmpleo=sumaEmpleo+float(eliminarPuntos(poblaciones['Unnamed: 3'][i]))
        if str(poblaciones['Unnamed: 2'][i]) == desempleo:

          sumaDesempleado=sumaDesempleado+float(eliminarPuntos(poblaciones['Unnamed: 3'][i]))

  empleados.append(sumaEmpleo)
  desempleado.append(sumaDesempleado)

In [34]:
# Se asignan los valores fecha, empleados y desempleados al nuevo dataset
df['Fecha'] = fechasList
df['Empleado'] = empleados
df['Desempleado'] = desempleado

import statistics

list =  [1, 2, 3, 4, 5, 6]
mean = statistics.mean(df['Desempleado'])
print(mean)

369816.68


In [12]:
'''
Esta fucion devuelve las posiciones en donde se encuentra la fila de la fechaInicio
y fechaFin ingresdas con el objetivo de indetificar el rango de fechas que se va a 
graficar

'''


def devolver_posiciones(fechaInicio, fechaFin, data):
  inicio=0
  fin=0
  aux=0
  for i in data.index:
    if data['Fecha'][i]==fechaInicio:
      inicio=i
      aux=i

    if data['Fecha'][i]==fechaFin:
      fin=i+1

  if fin < inicio:
    inicio=fin-1
    fin=aux+1
    
  return inicio, fin

In [13]:
'''
Este metodo filtra las fechas segun la fechainicio y fechafin en la data
'''
def filtrar_fechas(fechaInicio, fechaFin, data):
  inicio, fin =devolver_posiciones(fechaInicio, fechaFin, data)

  # retorna el dataset con las filas que se encuentren dentro del rango establecido por las pocisiones
  return data.iloc[inicio:fin]

## Reporte gráficas 

In [31]:
output = widgets.Output()
outout_plot = widgets.Output()
ALL = 'ALL'

#creo desplegable

dropdown_mesInicio = widgets.Dropdown(options = fechasList, description = 'Fecha Inicio')
dropdown_mesFinal = widgets.Dropdown(options = fechasList, description = 'Fecha Fin')

#funcion filtrar df & plot

def filtro (Fecha_Inicio, Fecha_Fin):
    output.clear_output()
    outout_plot.clear_output()
    
    d=filtrar_fechas('ene-21','dic-20',df)

    if (Fecha_Inicio == ALL) & (Fecha_Fin == ALL):
        filtro = df
    else:
        filtro = filtrar_fechas(Fecha_Inicio,Fecha_Fin,df)
        
    with output:
        display(filtro)
        
    with outout_plot:
        plt.figure (figsize= (10,12))
        plt.subplot(211)      
        plt.title('Empleados del pais')

        #plt.plot(d.Fecha, d.Empleado, 'b.-')
        plt.plot(filtro.Fecha, filtro.Empleado, 'b.-')

        plt.xticks(filtro.Fecha[::3].tolist())

        plt.xlabel('Fecha')
        plt.ylabel('Numero empleados')
        plt.legend()
        
        plt.subplot(212)
        plt.title('Desempleados del pais')

        plt.plot(filtro.Fecha, filtro.Desempleado, 'r.-')
        #plt.plot(d.Fecha, d.Desempleado, 'r.-')

        plt.xticks(filtro.Fecha[::3].tolist())

        plt.xlabel('Fecha')
        plt.ylabel('Numero desempleados')
        plt.legend()
        plt.show()
        
def dropdown_FechaInicio_eventhandler (change):
    filtro(change.new, dropdown_mesInicio.value)
    
def dropdown_FechaFin_eventhander(change):
    filtro(dropdown_mesFinal.value, change.new)
    
dropdown_mesFinal.observe(dropdown_FechaInicio_eventhandler, names ='value')
dropdown_mesInicio.observe (dropdown_FechaFin_eventhander, names = 'value')

# widgets layout
tab  = widgets.Tab([output, outout_plot])
tab.set_title(0,'Datos')
tab.set_title(1,'Gráficos')
dashboard = widgets.VBox([dropdown_mesInicio,dropdown_mesFinal, tab])
display(dashboard);

# Sectorización de empleo

## Preprocesamiento

In [16]:
#Se eliminan las filas con datos nulos y se elimna la columna 'Caracteristica'
personasSect=personasSect.dropna(how='all')
personasSect=personasSect.drop(['Característica'], axis=1)

In [17]:
# Se divide el dataset en subgrupos correespondientes a la sectrorizacion 
# Nacional, Urbano y Rural
nacional=personasSect.iloc[0:4]
urbano=personasSect.iloc[4:8]
rural=personasSect.iloc[8:12]

In [18]:
'''
Esta funcion retorna los sectores y los valores segun la fecha.
Alunos sectores no tienen valores por lo que toman el valor de 0, los que poseen valores 
como 45,89% se reemplaza la (',' - '.') y se le elimina el '%' para poder convertirlo a un valor 
de tipo float y realizar las graficas.

 '''
def filtrar_sectores(data,fecha):

  df = pd.DataFrame()
  df['Sector'] = None
  df['Porcentajes'] = None

  df['Sector']=data['Sector']
  valores=data[fecha]
  porcentajes=[]
  for x in valores:
    if x==0:
      porcentajes.append(0)
    elif(x=='-'):
      porcentajes.append(0)
    else:
      x=x.replace(",", ".")
      v=float(x.strip('%'))
      porcentajes.append(v)
  df['Porcentajes']=porcentajes
  return df

## Reporte gráficas

In [19]:
output = widgets.Output()
outout_plot = widgets.Output()
ALL = 'ALL'

#creo desplegable

dropdown_Fecha = widgets.Dropdown(options = fechasList, description = 'Fecha')


#funcion filtrar df & plot

def filtroPie(fecha,j):
    output.clear_output()
    outout_plot.clear_output()
    

    if fecha == ALL:
      print('')

    else:
        dataNacional = filtrar_sectores(nacional,fecha)
        dataUrbano = filtrar_sectores(urbano,fecha)
        dataRural = filtrar_sectores(rural,fecha)
        
    with output:
        display(filtroPie)
        
    with outout_plot:
        print('GRAFICAS PIE POR SECTORIZACION')
        plt.figure()
        colores = ['#A2C736', '#367BC7', '#7036C7','#E9EC21']
        plt.pie(dataNacional['Porcentajes'], labels=dataNacional['Sector'], colors=colores, autopct='%.2f %%')
        plt.title('Nacional'+' '+'Fecha'+' '+fecha)
        
        print(' ')
        plt.figure()
        colores = ['#A2C736', '#367BC7', '#7036C7','#E9EC21']
        plt.pie(dataUrbano['Porcentajes'], labels=dataUrbano['Sector'], colors=colores, autopct='%.2f %%')
        plt.title('Urbano'+' '+'Fecha'+' '+fecha)

        print(' ')
        plt.figure()
        colores = ['#A2C736', '#367BC7', '#7036C7','#E9EC21']
        plt.pie(dataRural['Porcentajes'], labels=dataRural['Sector'], colors=colores, autopct='%.2f %%')
        plt.title('Rural'+' '+'Fecha'+' '+fecha)
        plt.show()
        
def dropdown_Fecha_eventhandler (change):
    filtroPie(change.new, dropdown_Fecha.value)
    
    
dropdown_Fecha.observe(dropdown_Fecha_eventhandler, names ='value')


# widgets layout
tab  = widgets.Tab([outout_plot])

tab.set_title(1,'Gráficos')
dashboard = widgets.VBox([dropdown_Fecha,tab])
display(dashboard);

# Subempleo, empleo pleno y empleo no pleno 

## Preprocesamiento

In [20]:
#Se crean subggrupos para subempleo, empleo pleno, empleo no pleno
subempleoData = dataHist[dataHist[ 'Unnamed: 2' ] ==  "Subempleo"]
empleoPData = dataHist[dataHist[ 'Unnamed: 2' ] ==  "Empleo Adecuado/Pleno"]
emeploNpData= dataHist[dataHist[ 'Unnamed: 2' ] ==  "Otro Empleo no pleno"]

In [21]:

#Se crean nuevos datasets de cada uno de los subgrupos asignandoles nombres en las columnas
subempleosNum = pd.DataFrame()
subempleosNum['Fecha'] = subempleoData['Unnamed: 1']
subempleosNum['Indicadores'] = subempleoData['Unnamed: 2']
subempleosNum['Total'] = subempleoData['Unnamed: 3']

empleoPlenoNum = pd.DataFrame()
empleoPlenoNum['Fecha'] = empleoPData['Unnamed: 1']
empleoPlenoNum['Indicadores'] = empleoPData['Unnamed: 2']
empleoPlenoNum['Total'] = empleoPData['Unnamed: 3']

empleoNplenoNum = pd.DataFrame()
empleoNplenoNum['Fecha'] = emeploNpData['Unnamed: 1']
empleoNplenoNum['Indicadores'] = emeploNpData['Unnamed: 2']
empleoNplenoNum['Total'] = emeploNpData['Unnamed: 3']

In [22]:
#Retorna un dataset con ordenado por el mes
def return_data_odernado(data):
  totales=[]
  fechas=[]
  ind=[]

  df = pd.DataFrame()
  df['Fecha'] = None
  df['Indicadores'] = None
  df['Total'] = None

  for j in fechasList:
    for i in data.index:
      if str(data['Fecha'][i]) == j:

          if str(data['Indicadores'][i]) == 'Subempleo':
            totales.append(float(eliminarPuntos(data['Total'][i])))
            fechas.append(data['Fecha'][i])
            ind.append(data['Indicadores'][i])

          if str(data['Indicadores'][i]) == 'Otro Empleo no pleno':
            totales.append(float(eliminarPuntos(data['Total'][i])))
            fechas.append(data['Fecha'][i])
            ind.append(data['Indicadores'][i])

          if str(data['Indicadores'][i]) == 'Empleo Adecuado/Pleno':
            totales.append(float(eliminarPuntos(data['Total'][i])))
            fechas.append(data['Fecha'][i])
            ind.append(data['Indicadores'][i])

  df['Fecha']=fechas
  df['Indicadores']=ind
  df['Total']=totales
  return df

In [23]:
#se eliminan las fechas repetidas 
def filtrar_anio(data):
    #Se liminan las fechas repetidas
  ordenadaLista=[]
  anioLista=[]
  fechascomparacion=data['Fecha']
  for i in fechascomparacion:
    anioLista.append('20'+i[4:6])


  for element in anioLista:
      if element not in ordenadaLista:
          ordenadaLista.append(element)

  ordenados = sorted(ordenadaLista)

  return ordenados

In [24]:
#nuevos datasets sin fechas repetidas
s=return_data_odernado(subempleosNum)
ep=return_data_odernado(empleoPlenoNum)
emnp=return_data_odernado(empleoNplenoNum)

In [25]:
#se suman los valores totales de subempleo, empleo pleno y empleo no pleno segun el año 
def sumar_valores_anio(data):
  df = pd.DataFrame()
  df['Anio'] = None
  df['Indicadores'] = None
  df['Total'] = None
  total=0
  anios=[]
  totales=[]
  ind=[]
  f=filtrar_anio(data)
  label=''
  
  for j in f:
    
    total=0
    for i in data.index:
      if str('20'+data['Fecha'][i][4:6]) == j:
         total = total+data['Total'][i]
         label=data['Indicadores'][i]
 
    anios.append(j)
    totales.append(total)
    ind.append(label)
  
  df['Total']=totales
  df['Anio']=anios 
  df['Indicadores']=ind 

  return df

In [26]:
# datasets con los totales por año
dataS=sumar_valores_anio(s)
dataEP=sumar_valores_anio(ep)
dataENP=sumar_valores_anio(emnp)

In [27]:
# devuelve los datos de un dataset segun el rango de años ingresado
def devolver_posiciones_anio(anioIncio, anioFin, data):
  inicio=0
  fin=0
  aux=0
  for i in data.index:
    if data['Anio'][i]==anioIncio:
      inicio=i
      aux=i

    if data['Anio'][i]==anioFin:
      fin=i+1

  if fin < inicio:
    inicio=fin-1
    fin=aux+1
    
  return inicio, fin

In [28]:
# obtencion de los datos segun el rando de años
def obtener_rango_anios(anioIncio, anioFin, data):
  inicio, fin =devolver_posiciones_anio(anioIncio, anioFin, data)

  # retorna el dataset con las filas que se encuentren dentro del rango establecido por las pocisiones
  return data.iloc[inicio:fin]

## Reporte gráficas

In [32]:
output = widgets.Output()
outout_plot = widgets.Output()
ALL = 'ALL'

#creo desplegable

dropdown_anioInicio = widgets.Dropdown(options = dataS['Anio'], description = 'Año Inicio')
dropdown_anioFin = widgets.Dropdown(options = dataS['Anio'], description = 'Año Fin')

#funcion filtrar df & plot

def filtroHistA (anio_Inicio, anio_Fin):
    output.clear_output()
    outout_plot.clear_output()
    

    if (anio_Inicio == ALL) & (anio_Fin == ALL):
        filtro = df
    else:

        datosGS = obtener_rango_anios(anio_Inicio,anio_Fin,dataS)
        datosGEP = obtener_rango_anios(anio_Inicio,anio_Fin,dataEP)
        datosGENP = obtener_rango_anios(anio_Inicio,anio_Fin,dataENP)

        
    with output:
        display(filtroHistA)
        
    with outout_plot:
      # set width of bar
      barWidth = 0.3
      fig = plt.subplots(figsize =(18, 8))
      
      # set height of bar
      IT = np.array(datosGS["Total"])
      ECE = np.array(datosGEP["Total"])
      CSE = np.array(datosGENP["Total"])
      
      # Set position of bar on X axis
      br1 = np.arange(len(IT))
      br2 = [x + barWidth for x in br1]
      br3 = [x + barWidth for x in br2]

      # Make the plot
      plt.bar(br1, IT, color ='r', width = barWidth,
              edgecolor ='grey', label ='Subempleo')
      plt.bar(br2, ECE, color ='g', width = barWidth,
              edgecolor ='grey', label ='Empleo Pleno')
      plt.bar(br3, CSE, color ='b', width = barWidth,
              edgecolor ='grey', label ='Empleo No pleno')
      
      # Adding Xticks
      plt.xlabel('Año', fontweight ='bold', fontsize = 15)
      plt.ylabel('Datos en Millones', fontweight ='bold', fontsize = 15)
      plt.title('Personas en Subempleo, Empleo pleno y empleo no pleno')
      plt.xticks([r + barWidth for r in range(len(IT))],
              np.array(datosGS["Anio"]))
      
      plt.legend()
      plt.show()
        
def dropdown_AnioInicio_eventhandler (change):
    filtroHistA( dropdown_anioInicio.value, change.new)
    
def dropdown_AnioFin_eventhander(change):
    filtroHistA(change.new,dropdown_anioFin.value)
    
dropdown_anioFin.observe(dropdown_AnioInicio_eventhandler, names ='value')
dropdown_anioInicio.observe (dropdown_AnioFin_eventhander, names = 'value')

# widgets layout
tab  = widgets.Tab([outout_plot])
tab.set_title(1,'Gráficos')
dashboard = widgets.VBox([dropdown_anioInicio,dropdown_anioFin, tab])
display(dashboard);

# Estadística descriptiva 

Describir con estadística descriptiva los datos que se encuentran en los archivos

## Poblaciones
Dentro del archivo 'poblaciones.csv'.
Segun los datos presentados el numero de mujeres es mayor al de los hombres pero estos ocupan mas puestos de trabajo que las mujeres, hasta octubre del 2021 la poblacion total alcanza los 17 897 230 de habitantes en el pais con 9 137 335 mujeres y 8 759 895 con una diferencia del 4.13%.



Empleados
* Promedio de personas con empleo desde el 2007 = 7 295 030

Desempleados
* Promedio de personas desempleadas desde el 2007 = 369 816



## Sectorizacióm de empleo
Se puede observar que desde el año 2007 el sector Formal Urbano ha mantenido el mayor numero de empleados con un promedio del 57.6 %.

El sector con menos numero de empleados desde el 2007 es el sector del empleo doméstico rural con un promedio del 1.44 %

# Concluciones 
Los dos datasets se tuvieron que preprocesar para presentar de una manera mas legible los datos en las gráficas ya que existían datos nulos, dichos datos fueron reemplazados con un valor de 0 para que no obstruyan el procesamiento de las demás filas.

Las graficas generadas son interactivas a las cuales se les pueden asignar valores según el mes y el año y se presentará la distribución de los datos según el rango especificado.

Existe una gran diferencia entre el numero de empleados y desempleados, segun las graficas presentadas desde diciembre del 2007 hasta octubre del 2021, sin embargo el numero de empleados ha variado mucho a lo largo de los años, en diciembre del 2007 el numero de empleados alcanzo los 6 019 332 siendo el año con el menor numero de empleados hasta la fecha, hasta octubre del año 2021 se alcanzo el mayor numero de empleados con 8 066 696.

En cuanto a la sectorización del empleo el sector formal urbano posee el mayor numero de empleados desde el año 2007 hasta la actualidad.

Los histogramas presentan una mayor distribucion de personas con un empleo pleno desde al año 2007 hasta la actualidad.

